## Combine the different days data

In [3]:
import pandas as pd
import os

directory = './Twitter_Airline Dataset'
all_files = [file for file in os.listdir(directory) if file.endswith('.csv')]

all_dataframes = []
for file in all_files:
    try:
        df = pd.read_csv(os.path.join(directory, file), encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(os.path.join(directory, file), encoding='ISO-8859-1')  
    all_dataframes.append(df)

combined_dataframe = pd.concat(all_dataframes, ignore_index=True)
combined_csv_path = os.path.join(directory, 'combined_csv.csv')
combined_dataframe.to_csv(combined_csv_path, index=False)

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("TwitterAirlineAnalysis").getOrCreate()

df = spark.read.csv("./Twitter_Airline Dataset/combined_csv.csv", header=True)

df.show()


+--------------------+------------------+---------------+----------+-------+--------------------+--------+-----------+------+----------+--------+-------+--------------------+---------------+-----------------+--------------+--------------+----------------------+----------+-------------------+-------------+--------------------+-----------+---------------+--------+--------------+--------------------+
|            _unit_id|       _created_at|        _golden|       _id|_missed|         _started_at|_tainted|   _channel|_trust|_worker_id|_country|_region|               _city|            _ip|airline_sentiment|negativereason|       airline|airline_sentiment_gold|      name|negativereason_gold|retweet_count|                text|tweet_coord|  tweet_created|tweet_id|tweet_location|       user_timezone|
+--------------------+------------------+---------------+----------+-------+--------------------+--------+-----------+------+----------+--------+-------+--------------------+---------------+--------

In [7]:
df.count()

587942

Task 1: To list and justify your interpretation of the dataset. 
For example 
1) what are the meanings of the columns being used, how and why are they being used; 
2) list (if any) thepre-processing on the dataset like deduplicating, filling up missing values.

## Deduplicate based on uniqiue identifiers

In [8]:
df_deduplicated = df.drop_duplicates(subset=['_unit_id', '_id'])

In [9]:
df_deduplicated.count()

56022

## Missing Value

Fill missing 'retweet count' with mean

In [10]:
from pyspark.sql.functions import when, col, mean, lit
mean_val = df_deduplicated.select(mean(df_deduplicated['retweet_count'])).collect()[0][0]
mean_val

0.08358175375442939

In [11]:
df_deduplicated = df_deduplicated.withColumn('retweet_count', when(col('retweet_count').isNull(), mean_val).otherwise(col('retweet_count')))

In [13]:
df_deduplicated.show()

+--------------------+------------------+---------------+----------+------------------+--------------------+--------+-----------+------+----------+--------+-------+-----------+---------------+-----------------+--------------+--------------+----------------------+---------------+-------------------+-------------------+--------------------+--------------------+---------------+--------+--------------------+--------------------+
|            _unit_id|       _created_at|        _golden|       _id|           _missed|         _started_at|_tainted|   _channel|_trust|_worker_id|_country|_region|      _city|            _ip|airline_sentiment|negativereason|       airline|airline_sentiment_gold|           name|negativereason_gold|      retweet_count|                text|         tweet_coord|  tweet_created|tweet_id|      tweet_location|       user_timezone|
+--------------------+------------------+---------------+----------+------------------+--------------------+--------+-----------+------+------

Fill missing 'user_timezone' with mode

In [22]:
df_deduplicated = df_deduplicated.withColumn("user_timezone", when(col("user_timezone") == "NULL", None).otherwise(col("user_timezone")))

# mode of 'user_timezone'
mode_timezone = df_deduplicated.na.drop(subset=["user_timezone"]) \
                               .groupBy("user_timezone") \
                               .count() \
                               .orderBy("count", ascending=False) \
                               .first()["user_timezone"]

# fill missing values with the mode
df_deduplicated = df_deduplicated.na.fill({"user_timezone": mode_timezone})

# Count the number of rows that were filled
filled_rows_count = df_deduplicated.filter(col("user_timezone") == mode_timezone).count()

mode_timezone, filled_rows_count


('Eastern Time (US & Canada)', 34084)

In [21]:
df_deduplicated.show()

+--------------------+------------------+---------------+----------+------------------+--------------------+--------+-----------+------+----------+--------+-------+-----------+---------------+-----------------+--------------+--------------+----------------------+---------------+-------------------+-------------------+--------------------+--------------------+---------------+--------+--------------------+--------------------+
|            _unit_id|       _created_at|        _golden|       _id|           _missed|         _started_at|_tainted|   _channel|_trust|_worker_id|_country|_region|      _city|            _ip|airline_sentiment|negativereason|       airline|airline_sentiment_gold|           name|negativereason_gold|      retweet_count|                text|         tweet_coord|  tweet_created|tweet_id|      tweet_location|       user_timezone|
+--------------------+------------------+---------------+----------+------------------+--------------------+--------+-----------+------+------

## TASK 2

In [24]:
unique_negative_reasons = df_deduplicated.select("negativereason").distinct()
unique_negative_reasons.show(truncate=False)

+--------------+
|negativereason|
+--------------+
|longlines     |
|cancel        |
|lostluggae    |
|late          |
|booking       |
|CSProblem     |
|badflight     |
|canttell      |
|airplanestaff |
|damagedluggage|
|NULL          |
+--------------+



In [26]:
uique_airlines = df_deduplicated.select("airline").distinct()
uique_airlines.show(truncate=False)

+--------------+
|airline       |
+--------------+
|Delta         |
|Virgin America|
|United        |
|US Airways    |
|Southwest     |
|American      |
|NULL          |
+--------------+



In [28]:
null_count = df_deduplicated.filter(col("airline").isNull()).count()
null_string_count = df_deduplicated.filter(col("airline") == "NULL").count()
total_nulls = null_count + null_string_count
total_nulls

239

Since it is only 239 rows of 56022 imma drop that shit

In [29]:
df_deduplicated = df_deduplicated.filter((col("airline").isNotNull()) & (col("airline") != "NULL"))

In [31]:
from pyspark.sql.functions import count, collect_list, udf
from pyspark.sql.types import ArrayType, StringType

def top_n_reasons(reasons, counts, n=5):
    return [reason for reason, count in sorted(zip(reasons, counts), key=lambda x: x[1], reverse=True)[:n]]

top_n_udf = udf(top_n_reasons, ArrayType(StringType()))

top_neg_reasons = df_deduplicated.groupBy('airline', 'negativereason') \
    .agg(count('negativereason').alias('reason_count'))

# Group by airline again to collect all reasons and counts into lists
airline_reasons = top_neg_reasons.groupBy('airline') \
    .agg(collect_list('negativereason').alias('all_reasons'), 
         collect_list('reason_count').alias('all_counts'))

# top 5 reasons for each airline
top_5_neg_reasons = airline_reasons.withColumn('top_5_reasons', top_n_udf('all_reasons', 'all_counts'))

top_5_neg_reasons.select('airline', 'top_5_reasons').show(truncate=False)

+--------------+---------------------------------------------------------------+
|airline       |top_5_reasons                                                  |
+--------------+---------------------------------------------------------------+
|Delta         |[damagedluggage, lostluggae, airplanestaff, booking, longlines]|
|Virgin America|[late, damagedluggage, booking, airplanestaff, cancel]         |
|United        |[CSProblem, booking, canttell, badflight, damagedluggage]      |
|US Airways    |[late, canttell, cancel, CSProblem, booking]                   |
|Southwest     |[CSProblem, cancel, canttell, badflight, lostluggae]           |
|American      |[late, cancel, canttell, airplanestaff, lostluggae]            |
+--------------+---------------------------------------------------------------+

